In [1]:
#!pip freeze > requirements.txt

In [2]:
!pip install PySide2
!pip install scipy
!pip install numpy
!pip install tqdm
!pip install mayavi

  Using cached PySide2-5.15.2.1-5.15.2-cp35.cp36.cp37.cp38.cp39.cp310-none-win_amd64.whl (137.4 MB)
  Using cached shiboken2-5.15.2.1-5.15.2-cp35.cp36.cp37.cp38.cp39.cp310-none-win_amd64.whl (2.3 MB)
     ---------------------------------------- 0.0/42.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/42.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/42.5 MB ? eta -:--:--
     ---------------------------------------- 0.1/42.5 MB 1.1 MB/s eta 0:00:39
     ---------------------------------------- 0.1/42.5 MB 1.1 MB/s eta 0:00:39
     ---------------------------------------- 0.1/42.5 MB 1.1 MB/s eta 0:00:39
     ---------------------------------------- 0.1/42.5 MB 1.1 MB/s eta 0:00:39
     --------------------------------------- 0.2/42.5 MB 621.6 kB/s eta 0:01:09
     --------------------------------------- 0.2/42.5 MB 621.6 kB/s eta 0:01:09
     --------------------------------------- 0.2/42.5 MB 621.6 kB/s eta 0:01:09
     -----------------------

  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached mayavi-4.8.1-py3-none-any.whl
  Using cached envisage-6.1.1-py3-none-any.whl (281 kB)
  Using cached traitsui-7.4.3-py3-none-any.whl (1.5 MB)
  Using cached traits-6.4.1-cp310-cp310-win_amd64.whl (5.0 MB)
  Using cached vtk-9.2.6-cp310-cp310-win_amd64.whl (48.8 MB)
  Using cached pyface-7.4.4-py3-none-any.whl (1.3 MB)
  Using cached apptools-5.2.0-py3-none-any.whl (229 kB)
  Using cached configobj-5.0.8-py2.py3-none-any.whl (36 kB)
     ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
     ---------------------------------------- 0.1/7.6 MB 544.7 kB/s eta 0:00:14
      --------------------------------------- 0.1/7.6 MB 1.1 MB/s eta 0:00:07
      --------------------------------------- 0.1/7.6 MB 1.1 MB/s eta 0:00:07
     - -------------------------------------- 0.2/7.6 MB 1.0 MB/s eta 0:00:08
     - ------------------------------

In [3]:
import sys, os, importlib

In [4]:
names = ['numpy', 'scipy', 'matplotlib', 'tqdm', 'PySide2', 'mayavi']
def check_modules(module_names):
    for name in module_names:
        if name in sys.modules:
            print(f"{name!r} already in sys.modules")
        elif (spec := importlib.util.find_spec(name)) is not None:
            # If you choose to perform the actual import ...
            module = importlib.util.module_from_spec(spec)
            sys.modules[name] = module
            spec.loader.exec_module(module)
            print(f"{name!r} has been imported")
        else:
            print(f"can't find the {name!r} module")
check_modules(names)

'numpy' has been imported
'scipy' has been imported
'matplotlib' has been imported
'tqdm' has been imported
'PySide2' has been imported
'mayavi' has been imported


Если какой-то модуль не установлен, то нужно его установить через !pip intsall package. Скорее всего это будет PySide2.
Вроде я указал все, что нужно

In [5]:
import numpy as np
from scipy.integrate import odeint
from tqdm import tqdm
from mayavi.mlab import quiver3d
from mayavi import mlab
from scipy.special import lpmv, eval_legendre
from typing import Union

In [6]:
g_1_0 = -29404.8 * 1e-9
g_1_1 = -1450.9 * 1e-9
h_1_1 = 4652.5 * 1e-9
g_2_0 = -2499.6 * 1e-9
g_2_1 = 2982.0 * 1e-9
h_2_1 = -2991.6 * 1e-9
g_2_2 = 1677.0 * 1e-9
h_2_2 = -734.6 * 1e-9
Re = 6400 * 1e3
Rm = 16 * Re
b = 5 * Re
mu0=1.25663706*1e-6
m_x = 4*np.pi*Re**3/mu0*g_1_1
m_y = 4*np.pi*Re**3/mu0*h_1_1
m_z = 4*np.pi*Re**3/mu0*g_1_0
Q_xx = 4*np.pi*Re**4/mu0*(-g_2_0 + np.sqrt(3) * g_2_2)
Q_yy = 4*np.pi*Re**4/mu0*(-g_2_0 - np.sqrt(3) * g_2_2)
Q_xy = 4*np.pi*np.sqrt(3)*Re**4/mu0*h_2_2
Q_xz = 4*np.pi*np.sqrt(3)*Re**4/mu0*g_2_1
Q_yz = 4*np.pi*np.sqrt(3)*Re**4/mu0*h_2_1

In [7]:
class Magnetic_field():
    
    def __init__(self, cf: bool=True, dip: bool=True, qp: bool=True, qp_sym: bool=True):
        self.cf = cf
        self.dip = dip
        self.qp = qp
        self.qp_sym = qp_sym
        if self.qp_sym and self.qp != True:
            self.qp = True
        
    # Поле Земли из диссертации 2004го года
    @staticmethod
    def B_earth_m(x: Union[int, float], y: Union[int, float], z: Union[int, float]):
        #global b, mu0, m_x, m_y, m_z, Q_xx, Q_yy, Q_xy, Q_xz, Q_yz
        global b, mu0, g_1_0, g_1_1, h_1_1, g_2_0, g_2_2, g_2_1, h_2_1, h_2_2
        m_x = 4*np.pi*Re**3/mu0*g_1_1
        m_y = 4*np.pi*Re**3/mu0*h_1_1
        m_z = 4*np.pi*Re**3/mu0*g_1_0
        Q_xx = 4*np.pi*Re**4/mu0*(-g_2_0 + np.sqrt(3) * g_2_2)
        Q_yy = 4*np.pi*Re**4/mu0*(-g_2_0 - np.sqrt(3) * g_2_2)
        Q_xy = 4*np.pi*np.sqrt(3)*Re**4/mu0*h_2_2
        Q_xz = 4*np.pi*np.sqrt(3)*Re**4/mu0*g_2_1
        Q_yz = 4*np.pi*np.sqrt(3)*Re**4/mu0*h_2_1
        r = np.sqrt(x**2+y**2+(z+b)**2)
        a_d = -m_x*(z+b)+m_y*y+m_z*x
        a_q = Q_xx*(z+b)**2+Q_yy*y**2-(Q_xx+Q_yy)*x**2-2*Q_xy*(z+b)*y-2*Q_xz*(z+b)*x+2*Q_yz*x*y
        B_x = m_z/r**3-3*x*a_d/r**5-((Q_xx+Q_yy)*x+Q_xz*(z+b)-Q_yz*y)/r**5-5*x*a_q/(2*r**7)
        B_y = m_y/r**3-3*y*a_d/r**5+(Q_yy*y-Q_xy*(z+b)+Q_yz*x)/r**5-5*y*a_q/(2*r**7)
        B_z = -m_x/r**3-3*(z+b)*a_d/r**5+(Q_xx*(z+b)-Q_xy*y-Q_xz*x)/r**5-5*(z+b)*a_q/(2*r**7)
        return -mu0/(4*np.pi)*np.array([B_x, B_y, B_z])
    
    # Поле дипольной компоненты
    @staticmethod
    def B_dip_e(rr: Union[int, float], tt: Union[int, float], pp: Union[int, float], x: Union[int, float], y: Union[int, float], z: Union[int, float]):
        global b, mu0, g_1_0, g_1_1, h_1_1
        r = np.sqrt(x**2+y**2+(z+b)**2)
        a_d = -m_x*(z+b)+m_y*y+m_z*x
        B_x = m_z/r**3-3*x*a_d/r**5
        B_y = m_y/r**3-3*y*a_d/r**5
        B_z = -m_x/r**3-3*(z+b)*a_d/r**5
        return -mu0/(4*np.pi)*np.array([B_x, B_y, B_z])
    
    # Поле симметричного квадруполя
    @staticmethod
    def B_qp_e_symm(rr: Union[int, float], tt: Union[int, float], pp: Union[int, float],
                    x: Union[int, float], y: Union[int, float], z: Union[int, float]):
        global b, mu0, Q_xx, Q_yy, Q_xy, Q_xz, Q_yz
        r = np.sqrt(x**2+y**2+(z+b)**2)
        Q_xx_ = -4*np.pi*Re**4/mu0*g_2_0
        a_q = Q_xx_*((z+b)**2+y**2-2*x**2)
        B_x = -(Q_xx_+Q_xx_)*x/r**5-5*x*a_q/(2*r**7)
        B_y = Q_xx_*y/r**5-5*y*a_q/(2*r**7)
        B_z = Q_xx_*(z+b)/r**5-5*(z+b)*a_q/(2*r**7)
        return -mu0/(4*np.pi)*np.array([B_x, B_y, B_z])
    
    # Поле квадруполя
    @staticmethod
    def B_qp_e(rr: Union[int, float], tt: Union[int, float], pp: Union[int, float],
               x: Union[int, float], y: Union[int, float], z: Union[int, float]):
        global b, mu0, Q_xx, Q_yy, Q_xy, Q_xz, Q_yz
        r = np.sqrt(x**2+y**2+(z+b)**2)
        a_q = Q_xx*(z+b)**2+Q_yy*y**2-(Q_xx+Q_yy)*x**2-2*Q_xy*(z+b)*y-2*Q_xz*(z+b)*x+2*Q_yz*x*y
        B_x = -((Q_xx+Q_yy)*x+Q_xz*(z+b)-Q_yz*y)/r**5-5*x*a_q/(2*r**7)
        B_y = (Q_yy*y-Q_xy*(z+b)+Q_yz*x)/r**5-5*y*a_q/(2*r**7)
        B_z = (Q_xx*(z+b)-Q_xy*y-Q_xz*x)/r**5-5*(z+b)*a_q/(2*r**7)
        return -mu0/(4*np.pi)*np.array([B_x, B_y, B_z])
    
    def B(self):
        if self.dip:
            if self.cf:
                return self.B_cfi_2004_plus_earth_dip
            else:
                return self.B_dip_e
        if self.qp_sym:
            if self.cf:
                return self.B_cfi_2004_plus_earth_qp_symm
            else:
                return self.B_qp_e_symm
        
    # Преобразование из сферической системы координат в декртову
    @staticmethod
    def transform(theta: Union[int, float], phi: Union[int, float]):
        C = np.array([[np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), np.cos(theta)],
                      [np.cos(theta)*np.cos(phi), np.cos(theta)*np.sin(phi), -np.sin(theta)],
                      [-np.sin(phi), np.cos(phi), 0]])
        return C

    # Вычисляем суммарное поле(cf+earth)
    def B_xyz_m(self, x: Union[int, float], y: Union[int, float], z: Union[int, float]):
        dip = self.dip
        qp = self.qp
        qp_sym = self.qp_sym
        r = np.sqrt(x**2+y**2+z**2)
        theta = np.arccos(z / r)
        phi = np.arctan(y / x) if x > 0 else np.arctan(y / x) + np.pi
        if self.dip:
            if self.qp:
                B_earth = self.B_dip_e(x, y, z) + self.B_qp_e(x, y, z)
            else:
                B_earth = self.B_dip_e(x, y, z)
        elif self.qp_sym:
            B_earth = self.B_qp_e(x, y, z, symmetric=True)
        else:
            B_earth = self.B_qp_e(x, y, z, symmetric=False)
        #B_earth = self.B_earth_m(x, y, z)
        if self.cf:
            B_cf_sphere = self.B_cfi_2004(r, theta, phi)
            C = self.transform(theta, phi)
            B_cf = B_cf_sphere @ C
            return B_earth + B_cf
        else:
            return B_earth

    # Строим сферу
    @staticmethod
    def plot_sphere(r: Union[int, float]=6400*1e3, x_0: Union[int, float]=0, y_0: Union[int, float]=0, z_0: Union[int, float]=-b):
        "Вообще сфера получается повернутой, потом надо будет ее перевернуть, чтобы выглядело красивее"
        [phi, theta] = np.mgrid[0:2 * np.pi:30j, 0:np.pi:30j]
        x = r * np.cos(phi) * np.sin(theta)
        y = r * np.sin(phi) * np.sin(theta)
        z = r * np.cos(theta)
        mlab.mesh(x + x_0, y + y_0, z + z_0, colormap='gist_earth')
    
    # Считаем магнитные линии
    def magn_lines_m(self, start_points: list, step: Union[int, float]=1e5, R_bound: Union[int, float]=6400*1e3*80, max_iter: int=10_000):
        global Re
        lines = []
        mag_field = self.B()
        for point in tqdm(start_points, desc='Lines plotted'):
            r_gse, theta_gse, phi_gse = point
            x_gse = r_gse * np.sin(theta_gse) * np.cos(phi_gse)
            y_gse = r_gse * np.sin(theta_gse) * np.sin(phi_gse)
            z_gse = r_gse * np.cos(theta_gse)
            x = z_gse
            y = y_gse
            z = -(x_gse+b)
            r = np.sqrt(x**2+y**2+z**2)
            theta = np.arccos(z / r)
            phi = np.arctan(y / x) if x > 0 else np.arctan(y / x) + np.pi
            line = [np.array([x, y, z])]
            #B = self.B_xyz_m(x, y, z)
            B = mag_field(r, theta, phi, x, y, z)
            sign = -np.sign(x)
            i = 0
            while Re <= r_gse <= R_bound and i < max_iter:
                new_point = line[-1] + sign * step * B / np.linalg.norm(B)
                x_gse, y_gse, z_gse = -(new_point[2]+b), new_point[1], new_point[0]
                r_gse = np.sqrt(x_gse**2+y_gse**2+z_gse**2)
                line.append(new_point)
                #B = self.B_xyz_m(*new_point)
                r = np.sqrt(new_point[0]**2+new_point[1]**2+new_point[2]**2)
                theta = np.arccos(new_point[2] / r)
                phi = np.arctan(new_point[1] / new_point[0]) if new_point[0] > 0 else np.arctan(new_point[1] / new_point[0]) + np.pi
                B = mag_field(r, theta, phi, new_point[0], new_point[1], new_point[2])
                i += 1
            lines.append(line)
        return lines

    # Строим магнитные линии
    def plot_magnetic_field_lines_m(self, theta: Union[int, float], phi: Union[int, float]):
        global Re
        cf = self.cf
        if cf:
            color=(0.1, 0.3, 0.5) # синие для случая с учетом поля Чапмена_Ферраро
        else:
            color=(0.5, 0.3, 0.5) # красные для чистого поля Земли
        start_points = []
        for i in range(theta.shape[0]):
            for j in range(phi.shape[0]):
                theta_ = theta[i]
                phi_ = phi[j]
                point = np.array([Re, theta_, phi_])
                start_points.append(point)
        lines = self.magn_lines_m(start_points)
        for line in lines:
            line = np.array(line)
            mlab.plot3d(line[:, 0], line[:, 1], line[:, 2], tube_radius=None, color=color)
        self.plot_sphere()

    # Преобразование из системы Земли в смещенную систему координат
    @staticmethod
    def decart_to_m(r: Union[int, float], theta: Union[int, float], phi: Union[int, float]):
        "Этот метод уже нигде не используется, потому что было решено работать с смещенной системе координат, но вдруг потом где-то пригодится"
        global b
        x = r * np.sin(theta) * np.cos(phi)
        y = r * np.sin(theta) * np.sin(phi)
        z = r * np.cos(theta)
        x_m = z
        y_m = y
        z_m = -(x + b)
        r_m = np.sqrt(x_m**2 + y_m**2 + z_m**2)
        theta_m = np.arccos(z_m / r_m)
        phi_m = np.arctan(y_m / x_m) if x_m > 0 else np.arctan(y_m / x_m) + np.pi
        A = np.array([[np.sin(phi_m), -np.cos(phi_m), 0],
                      [np.cos(theta_m)*np.cos(phi_m), np.cos(theta_m)*np.sin(phi_m), -np.sin(theta_m)],
                      [np.sin(theta_m)*np.cos(phi_m), np.sin(theta_m)*np.sin(phi_m), np.cos(theta_m)]])
        return A, r_m, theta_m, phi_m
    
    #Полиномы Лежандра без фазы
    @staticmethod
    def P_n_m(n: int, m: int, x: Union[int, float]):
        "в диссертации полиномы Лежандра используются без фазы (-1)^m, учтем это"
        if m % 2 == 0:
            return lpmv(m, n, np.cos(x))
        else:
            return -lpmv(m, n, np.cos(x))
        
    #Производная от присоединенного полинома Лежандра в виде P_n_m(cos(x))
    def der_of_p_n_m(self, n: int, m: int, x: Union[int, float]):
        return (n*np.cos(x)*self.P_n_m(n, m, x)-(n+m)*self.P_n_m(n-1, m, x))/np.sin(x)
        
    def B_cfi_r(self, n: int, r: Union[int, float], theta: Union[int, float], phi: Union[int, float]):
        global b, Rm, Q_xx, Q_yy, Q_xy, Q_xz, Q_yz
        B_r = (n+1)*(-b/Rm)**(n+2)*(r/Rm)**(n-1)*(
                                                -n*(m_x+(n-1)/(4*b)*Q_xx)*self.P_n_m(n, 0, theta)+
                                                ((m_z+(n-1)/(3*b)*Q_xz)*np.cos(phi)+(m_y+(n-1)/(3*b)*Q_xy)*np.sin(phi))*self.P_n_m(n, 1, theta)+
                                                1/(6*b)*((0.5*Q_xx+Q_yy)*np.cos(2*phi)-Q_yz*np.sin(2*phi))*self.P_n_m(n, 2, theta)
                                                )
        return B_r
    
    def B_cfi_theta(self, n: int, r: Union[int, float], theta: Union[int, float], phi: Union[int, float]):
        global b, Rm, Q_xx, Q_yy, Q_xy, Q_xz, Q_yz
        B_theta = (n+1)/n*(-b/Rm)**(n+2)*(r/Rm)**(n-1)*(
                                            -n*(m_x+(n-1)/(4*b)*Q_xx)*self.der_of_p_n_m(n, 0, theta)+
                                            ((m_z+(n-1)/(3*b)*Q_xz)*np.cos(phi)+(m_y+(n-1)/(3*b)*Q_xy)*np.sin(phi))*self.der_of_p_n_m(n, 1, theta)+
                                            1/(6*b)*((0.5*Q_xx+Q_yy)*np.cos(2*phi)-Q_yz*np.sin(2*phi))*self.der_of_p_n_m(n, 2, theta)
                                                      )
        return B_theta
    
    def B_cfi_phi(self, n: int, r: Union[int, float], theta: Union[int, float], phi: Union[int, float]):
        global b, Rm, Q_xx, Q_yy, Q_xy, Q_xz, Q_yz
        B_phi = (n+1)/n*(-b/Rm)**(n+2)*(r/Rm)**(n-1)*(
                                                (-(m_z+(n-1)/(3*b)*Q_xz)*np.sin(phi)+(m_y+(n-1)/(3*b)*Q_xy)*np.cos(phi))*self.P_n_m(n, 1, theta)/np.sin(theta)+
                                                1/(3*b)*(-(0.5*Q_xx+Q_yy)*np.sin(2*phi)-Q_yz*np.cos(2*phi))*self.P_n_m(n, 2, theta)/np.sin(theta)
                                                     )
        return B_phi
    
    def B_cfi_2004(self, r: Union[int, float], theta: Union[int, float], phi: Union[int, float], N: int=1):
        '''r: радиус в смещенной системе координат
           theta: угол относительно оси z в смещенной системе координат
           phi: полярный угол в смещенной системе координат
           N: число членов ряда'''
        global b, mu0
        B_r = 0.0
        B_theta = 0.0
        B_phi = 0.0
        for n in range(1, N+1):
            B_r += self.B_cfi_r(n, r, theta, phi)
            B_theta += self.B_cfi_theta(n, r, theta, phi)
            B_phi += self.B_cfi_phi(n, r, theta, phi)
        return mu0/(4*np.pi*b**3)*np.array([B_r, B_theta, B_phi])
    
    def B_cfi_2004_plus_earth_dip(self, r: Union[int, float], theta: Union[int, float], phi: Union[int, float],
                                  x: Union[int, float], y: Union[int, float], z: Union[int, float], N: int=1):
        '''r: радиус в смещенной системе координат
           theta: угол относительно оси z в смещенной системе координат
           phi: полярный угол в смещенной системе координат
           N: число членов ряда'''
        global b, mu0
        B_r = 0.0
        B_theta = 0.0
        B_phi = 0.0
        #x = r * np.sin(theta) * np.cos(phi)
        #y = r * np.sin(theta) * np.sin(phi)
        #z = r * np.cos(theta)
        for n in range(1, N+1):
            B_r += self.B_cfi_r(n, r, theta, phi)
            B_theta += self.B_cfi_theta(n, r, theta, phi)
            B_phi += self.B_cfi_phi(n, r, theta, phi)
        B_cfi = mu0/(4*np.pi*b**3)*np.array([B_r, B_theta, B_phi])
        B_e_dip = self.B_dip_e(r, theta, phi, x, y, z)
        C = self.transform(theta, phi)
        B = B_cfi @ C + B_e_dip
        return B
    
    def B_cfi_2004_plus_earth_qp_symm(self, r: Union[int, float], theta: Union[int, float], phi: Union[int, float],
                                      x: Union[int, float], y: Union[int, float], z: Union[int, float], N: int=1):
        '''r: радиус в смещенной системе координат
           theta: угол относительно оси z в смещенной системе координат
           phi: полярный угол в смещенной системе координат
           N: число членов ряда'''
        global b, mu0
        B_r = 0.0
        B_theta = 0.0
        B_phi = 0.0
        #x = r * np.sin(theta) * np.cos(phi)
        #y = r * np.sin(theta) * np.sin(phi)
        #z = r * np.cos(theta)
        for n in range(1, N+1):
            B_r += self.B_cfi_r(n, r, theta, phi)
            B_theta += self.B_cfi_theta(n, r, theta, phi)
            B_phi += self.B_cfi_phi(n, r, theta, phi)
        B_cfi = mu0/(4*np.pi*b**3)*np.array([B_r, B_theta, B_phi])
        B_e_qp_symm = self.B_qp_e_symm(r, theta, phi, x, y, z)
        C = self.transform(theta, phi)
        B = B_cfi @ C + B_e_qp_symm
        return B

Посторим поле диполя

In [8]:
theta_1 = [np.pi / x for x in range(4, 11, 1)]
theta_2 = [np.pi/2 + np.pi / x for x in range(9, 34, 3)]
theta_qp = np.concatenate((theta_1, theta_2), axis=None)
theta_dip = np.array([x * np.pi / 12 for x in range(13)])
phi = np.array([x * np.pi / 6 for x in range(13)])
mf_with_cf_dip = Magnetic_field(cf=True, dip=True, qp=False, qp_sym=False)
mf_without_cf_dip = Magnetic_field(cf=False, dip=True, qp=False, qp_sym=False)
mf_with_cf_dip.plot_magnetic_field_lines_m(theta_dip, phi)
mf_without_cf_dip.plot_magnetic_field_lines_m(theta_dip, phi)
mlab.view(focalpoint=[0, 0, -b])
mlab.show()

Lines plotted: 100%|█████████████████████████████████████████████████████████████████| 169/169 [00:06<00:00, 26.30it/s]


In [9]:
mf_with_cf_qp = Magnetic_field(cf=True, dip=False, qp=True, qp_sym=True)
mf_without_cf_qp = Magnetic_field(cf=False, dip=False, qp=True, qp_sym=True)
mf_with_cf_qp.plot_magnetic_field_lines_m(theta_qp, phi)
mf_without_cf_qp.plot_magnetic_field_lines_m(theta_qp, phi)
mlab.view(focalpoint=[0, 0, -b])
mlab.show()

Lines plotted: 100%|████████████████████████████████████████████████████████████████| 208/208 [00:00<00:00, 323.67it/s]
